This is a slight modification of the word2vec tensorflow tutorial. This is labeled heavily so that the significance of each and every line is properly 
comprehended by the user.

In [46]:
import tensorflow as tf
import nltk
import urllib
import collections
import numpy as np
from six.moves import xrange

Defining a function to open the text file and create a list of words. The total number of unique words are also printed.

In [3]:
def read_data(filename = 'text8'):
    text_file = open('data/'+str(filename),'r')
    words = text_file.read().split()
    return words

words = read_data()
print(len(set(words))) # Prints the total number of unique words

count = collections.Counter(words) # To get the count of each words and visualize the top 10 most frequent words
count.most_common()[:10]

253854


[('the', 1061396),
 ('of', 593677),
 ('and', 416629),
 ('one', 411764),
 ('in', 372201),
 ('a', 325873),
 ('to', 316376),
 ('zero', 264975),
 ('nine', 250430),
 ('two', 192644)]

Defining a function to create data,count, word_dictionary and reversed dictionary. Rare words (words 
outside our definded vocabulary) are replaced with the 'UNK' token. 'word_dictionary' is a dictionary which maps the actual word to it's integer representation. 'count' will be a list, each element being the word and its number of occurrence.(First element of count is 'UNK' and the count of all words not present in the vocabulary that we define). 'data' is a list which is obtained by substituting each 
word in 'words' with its integer representation (the input data generated from read_data). So basically, we convert words to integer representation. 'reversed_dictionary' is the reverse mapping of 'word_dictionary' (from integer labels to words)

In [5]:
vocabulary_size = 50000

def create_train_data(words,vocab_size):
    # initially setting the first element of 'count' as the 'UNK' token (for rare words) and its occurrence as -1
    # Its occurence will be changed later
    count = [['UNK',-1]] 
    # Populating the count list with the rest of the words as per vocab_size. (We take the most common words)
    count.extend(collections.Counter(words).most_common(vocab_size-1)) # 1 less as the first element is 'UNK'
    
    # Now creating 'word_dictionary' which maps words to integer labels.
    word_dictionary = dict()
    for word,occurence in count:
        word_dictionary[word] = len(word_dictionary) # So it assigns an integer label as the dictionary fills up.
    
    # Now creating 'data' which converts words list to a list of its integer representations
    # We only take words defined in the word_dictionary as we have a fixed vocabulary(vocab_size)
    # All other words are 'UNK'
    data = list()
    unk_count = 0
    for word in words:
        if word in word_dictionary: 
            index = word_dictionary[word]
        else:
            index = 0 # Word not in dictionary, so label is 0 which is 'UNK'
            unk_count += 1 # To get the unknown count at the end to update the first element of 'count'
        data.append(index)
    
    # updating the first element of count. i.e, updating the count of 'UNK'
    count[0][1] = unk_count 
    # To get the reverse mapping
    reversed_dictionary = dict(zip(word_dictionary.values(),word_dictionary.keys())) 
    return data, count, word_dictionary, reversed_dictionary

# Print and check the visualize these objects
data, count, word_dictionary, reversed_dictionary = create_train_data(words,vocabulary_size)

Now we have our data prepared. Checking the first 10 elements of data and count.

In [6]:
data[:10], count[:10]

([5244, 3081, 12, 6, 195, 2, 3137, 46, 59, 156],
 [['UNK', 418391],
  ('the', 1061396),
  ('of', 593677),
  ('and', 416629),
  ('one', 411764),
  ('in', 372201),
  ('a', 325873),
  ('to', 316376),
  ('zero', 264975),
  ('nine', 250430)])

In [28]:
data_index = 0 
# This is a global variable and is declared as global in the function below to keep track of the index of data so as to generate the next batch.

def generate_batch(batch_size, num_skips, skip_window):
  global data_index
  assert batch_size % num_skips == 0
  assert num_skips <= 2 * skip_window
  batch = np.ndarray(shape=(batch_size), dtype=np.int32)
  labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
  span = 2 * skip_window + 1  # [ skip_window target skip_window ]
  buffer = collections.deque(maxlen=span)
  if data_index + span > len(data):
    data_index = 0
  buffer.extend(data[data_index:data_index + span])
  #print("Buffer")
  #print(buffer)
  data_index += span
  for i in range(batch_size // num_skips): 
      # basically i acts as a counter for taking the target word. Batch size is 8. 2 context words are taken,
      # 1 on the left and 1 on the right. So 8/2 = 4. Hence we get 4 target words. Uncomment the print statements to visualize. 
      
    context_words = [w for w in range(span) if w != skip_window]
    random.shuffle(context_words)
    words_to_use = collections.deque(context_words)
    #print("Words to use")
    #print(words_to_use)
    for j in range(num_skips):
      #print("prev Batch")
      #print(batch)
      batch[i * num_skips + j] = buffer[skip_window]  # Target Word. 
      # i*num_skips + j basically acts as a counter for the indices of the batch, just like writing numbers like 11 as (1*10 + 1)
      context_word = words_to_use.pop() 
      # Grab the index of a context word from words_to_use and then pop it from the deque. So now this popped index is stored in context_word
      labels[i * num_skips + j, 0] = buffer[context_word] # Now use this popped word  as the label. 
      # (We are predicting context words from center word)
      #print("Batch")
      #print(batch)
      #print("Labels")
      #print(labels)
    if data_index == len(data):
      buffer[:] = data[:span]
      data_index = span
    else:
      buffer.append(data[data_index])
      data_index += 1
  # Backtrack a little bit to avoid skipping words in the end of a batch
  data_index = (data_index + len(data) - span) % len(data)
  return batch, labels

Visualizing a batch of 8

In [29]:
batch, labels = generate_batch(batch_size=8, num_skips=2, skip_window=1)
for i in range(8):
  print(batch[i], reversed_dictionary[batch[i]],
        '->', labels[i, 0], reversed_dictionary[labels[i, 0]])

3081 originated -> 12 as
3081 originated -> 5244 anarchism
12 as -> 6 a
12 as -> 3081 originated
6 a -> 195 term
6 a -> 12 as
195 term -> 2 of
195 term -> 6 a


Bulding the skip-gram model and the tensorflow graph

In [32]:
batch_size = 128
embedding_size = 128  # Dimension of the embedding vector.
skip_window = 1       # How many words to consider left and right.
num_skips = 2         # How many times to reuse an input to generate a label. Use 2 when skip_window = 1 (So it generates 2 pairs for one word)

# We pick a random validation set to sample nearest neighbors. Here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent.
valid_size = 16     # Random set of words to evaluate similarity on.
valid_window = 100  # Only pick dev samples in the head of the distribution.
valid_examples = np.random.choice(valid_window, valid_size, replace=False) # generates a random sized subset in the range (o, valid_window)
num_sampled = 64    # Number of negative examples to sample.

In [40]:
vocabulary_size = 50000

graph = tf.Graph()
with graph.as_default():
     # setting placeholders and constants for input data, labels and validation dataset
    train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
    train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)
     
    # defining a variable for the word embeddings and initializing them with numbers between -1 and 1.
    embeddings = tf.Variable(tf.random_uniform([vocabulary_size, embedding_size],-1.0,1.0))
    embed = tf.nn.embedding_lookup(embeddings, train_inputs)  
    # This can be taken as the output of a multiplication happening at the first hidden layer of the neural net. (just for understanding)
    # Basically like a multiplication in which input tensor is a one hot. ( the hot element corresponding to the ids of words given by inputs)
    # But internally during backprop, its like a dictionary which reverse maps (during backprop) so that it sends 
    # out the gradients only to those rows of  embeddings that were used in that particular batch. Hence it will be a much more faster than
    # representing this operation with a typical neural network layer with the embeddings as its weights.
    # This picks up all the tensors in the embeddings according to the ids obtained from train_inputs
    # Remember that train_inputs is just a tensor of a set of integers governed by batch_size which are 
    # the integer representations of the words in data 
     
    # Constructing weights and biases for NCE Loss
    nce_weights = tf.Variable(tf.truncated_normal([vocabulary_size, embedding_size], stddev = 1.0/math.sqrt(embedding_size)))
    nce_biases = tf.Variable(tf.zeros([vocabulary_size]))
    # Compute the average NCE loss for the batch.
    # tf.nce_loss automatically draws a new sample of the negative labels each
    # time we evaluate the loss.
    # in NCE loss, 'inputs' actually represent the input to the nce_loss and not the input of the neural net.
    # The input to the nce_loss will be the output of the previous layer. The previous layer is the embedding_lookup layer
    #
    loss = tf.reduce_mean(
      tf.nn.nce_loss(weights=nce_weights,
                     biases=nce_biases,
                     labels=train_labels,
                     inputs=embed,
                     num_sampled=num_sampled,
                     num_classes=vocabulary_size))
    
    # Construct the SGD optimizer using a learning rate of 1.0.
    optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)
    # Compute the cosine similarity between minibatch examples and all embeddings.
    norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
    normalized_embeddings = embeddings / norm
    valid_embeddings = tf.nn.embedding_lookup(
          normalized_embeddings, valid_dataset)
    similarity = tf.matmul(
          valid_embeddings, normalized_embeddings, transpose_b=True)
    
    # Add variable initializer.
    init = tf.global_variables_initializer()

A high level explanation for the optimization of nce loss obtained from stackoverflow (given by curator23) : 
"The embeddings Tensor is your final output matrix. It maps words to vectors. Use this in your word prediction graph.
The input matrix is a batch of centre-word : context-word pairs (train_input and train_label respectively) generated from the training text.
While the exact workings of the nce_loss op are not yet know to me, the basic idea is that it uses a single layer network (parameters nce_weights and nce_biases) to map an input vector (selected from embeddings using the embed op) to an output word, and then compares the output to the training label (an adjacent word in the training text) and also to a random sub-sample (num_sampled) of all other words in the vocab, and then modifies the input vector (stored in embeddings) and the network parameters to minimise the error."

Training the skip-gram model

In [48]:
# Step 5: Begin training.
num_steps = 100001

with tf.Session(graph=graph) as session:
  # We must initialize all variables before we use them.
  init.run()
  print('Initialized')

  average_loss = 0
  for step in xrange(num_steps):
    batch_inputs, batch_labels = generate_batch(
        batch_size, num_skips, skip_window)
    feed_dict = {train_inputs: batch_inputs, train_labels: batch_labels}

    # We perform one update step by evaluating the optimizer op (including it
    # in the list of returned values for session.run()
    _, loss_val = session.run([optimizer, loss], feed_dict=feed_dict)
    average_loss += loss_val

    if step % 2000 == 0:
      if step > 0:
        average_loss /= 2000
      # The average loss is an estimate of the loss over the last 2000 batches.
      print('Average loss at step ', step, ': ', average_loss)
      average_loss = 0

    # Note that this is expensive (~20% slowdown if computed every 500 steps)
    if step % 10000 == 0:
      sim = similarity.eval()
      for i in xrange(valid_size):
        valid_word = reversed_dictionary[valid_examples[i]]
        top_k = 8  # number of nearest neighbors
        nearest = (-sim[i, :]).argsort()[1:top_k + 1]
        log_str = 'Nearest to %s:' % valid_word
        for k in xrange(top_k):
          close_word = reversed_dictionary[nearest[k]]
          log_str = '%s %s,' % (log_str, close_word)
        print(log_str)
  final_embeddings = normalized_embeddings.eval()

Initialized
Average loss at step  0 :  282.584777832
Nearest to nine: subsist, anisotropic, fellows, findable, freeciv, rumors, seventeenth, asn,
Nearest to UNK: acrimonious, oxen, qing, mkultra, literal, mutinied, meddle, mercenary,
Nearest to four: lifespan, advantages, dhimmi, eudoxia, tert, suzy, straightforwardly, ramesses,
Nearest to one: iucn, gamble, rf, elves, barton, anz, vitry, gilt,
Nearest to he: registration, local, paraphrased, rabinowitz, supers, gourmet, anatoly, endocrinology,
Nearest to by: prom, ehud, southern, assaulted, sectarianism, indochina, plunder, pledges,
Nearest to his: boo, radiosity, opus, jungles, unit, franconia, oceanography, color,
Nearest to no: uncontrolled, specially, sn, mouthpieces, plugs, foix, miles, lineage,
Nearest to its: crashed, sophistication, appearances, unicellular, analysing, gurion, humorous, chloride,
Nearest to often: crevasses, prosthetic, pompidou, outstripped, reconquest, tapes, lifespan, divided,


Nearest to th: weld, brunel, martins, point, dioxide, cezanne, observing, inwardly,
Nearest to their: anacreon, af, appendices, clip, marshalling, scenery, sharper, minarchists,
Nearest to been: adopts, immune, stark, christensen, peyton, datum, heterocyclic, soul,
Nearest to it: falls, drucker, runs, furor, sciences, louisiana, drivers, grotesque,
Nearest to there: nowell, guernica, withered, adolphus, triggering, sara, warns, lutheranism,
Nearest to when: several, disparagingly, sandro, serial, communism, calendars, warned, lacquer,


Average loss at step  2000 :  113.735357479


Average loss at step  4000 :  52.778852736


Average loss at step  6000 :  33.5482197268


...

Average loss at step  92000 :  4.67600551403


Average loss at step  94000 :  4.71477692652


Average loss at step  96000 :  4.69089664531


Average loss at step  98000 :  4.60148849851


Average loss at step  100000 :  4.69934381998
Nearest to nine: eight, seven, six, five, zero, four, three, ursus,
Nearest to UNK: kapoor, dasyprocta, ursus, mico, callithrix, operatorname, circ, peacocks,
Nearest to four: five, three, seven, eight, six, two, zero, kapoor,
Nearest to one: four, seven, two, five, six, three, kapoor, eight,
Nearest to he: it, she, they, who, there, we, never, but,
Nearest to by: be, was, as, operatorname, seizures, kapoor, busan, seven,
Nearest to his: their, her, its, the, my, s, thaler, our,
Nearest to no: it, unequal, sponsors, only, any, nine, franc, positions,
Nearest to its: their, his, the, her, thaler, jethro, analysing, some,
Nearest to often: sometimes, also, pulau, commonly, now, usually, generally, still,
Nearest to th: six, seven, accumulator, nine, eight, dioxide, three, brunel,
Nearest to their: its, his, her, the, some, pulau, agouti, thaler,
Nearest to been: be, was, were, reliance, become, by, had, sponsors,
Nearest to it: he, there, thi

In [50]:
final_embeddings[:10]

array([[ 0.00751682, -0.05371699, -0.07132055, ..., -0.13933519,
        -0.02045279, -0.04116968],
       [ 0.08291626,  0.11531038, -0.08103251, ...,  0.12472363,
         0.05915395, -0.06485777],
       [ 0.12645645,  0.12861994, -0.15893146, ...,  0.02102033,
        -0.08008344, -0.17346135],
       ..., 
       [ 0.07011682,  0.00313191, -0.05450524, ..., -0.07874514,
        -0.05461983, -0.1422379 ],
       [ 0.03490714, -0.02096882,  0.01477128, ..., -0.02078016,
        -0.04375534, -0.05463719],
       [ 0.06420515, -0.06192603, -0.09500519, ..., -0.0512049 ,
        -0.07853603,  0.0226666 ]], dtype=float32)